In [ ]:
import pandas as pd
import csv
import sys
from collections import Counter
import matplotlib.pyplot as plt
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
nltk.download('omw-1.4')
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, Flatten, MaxPooling1D
import numpy as np
from keras.preprocessing import text
from scipy.special import softmax
import random
import pickle
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import string
import nltk 
from nltk.corpus import stopwords
from string import punctuation
from os import listdir
import time

In [ ]:
#loading in Airlines data
data = pd.read_csv("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\Data\\AirlinesData.csv")
data.drop(data.columns[[0,3,4,5,6,7,8,9,11,12,13,14]], axis=1, inplace=True)
data = data[data.airline_sentiment_confidence == 1.0]
data = data[data.airline_sentiment != "neutral"]  # removing neutral sentiment rows
def preprocess(text):
    preprocessed_text = []
    for t in text.split():
        if len(t) > 1:
            t = '' if t[0] == '@' and t.count('@') == 1 else t  # removing links and mentions
            t = '' if t.startswith('http') else t
            #t = '' if t.startswith('#') else t
        preprocessed_text.append(t)
    return ' '.join(preprocessed_text)
Cleaned_Tweet = []
for tweet in data["text"]:
    cleaned = preprocess(tweet)
    tokenizedtweet = tweet.split()  # Tokenizing tweets
    #tweetNoStopWords = [word for word in tokenizedtweet if word not in stopwords.words('english')] # Removing stopwords
    #cleaned = ' '.join(tweetNoStopWords) # Joining tokenized tweet
    cleaned = cleaned.translate(str.maketrans('', '', string.punctuation)) # Removing punctuation
    #cleaned = ''.join([i for i in cleaned if not i.isdigit()]) # removing digits
    Cleaned_Tweet.append(cleaned)

# producing test/training data 
data["Cleaned_Tweet"] = Cleaned_Tweet
dataShuffled = data.sample(frac=1)
ShuffledTweets = []
for tweet in dataShuffled["Cleaned_Tweet"]:
    ShuffledTweets.append(tweet)
ShuffledSenti = []
for senti in dataShuffled["airline_sentiment"]:
    ShuffledSenti.append(senti)    
train_x = ShuffledTweets[:7118] # list of cleaned tweets
test_x = ShuffledTweet

In [ ]:
# load doc into memory
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text
 
# turn a doc into clean tokens
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    #stop_words = set(stopwords.words('english'))
    #tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    #tokens = [word for word in tokens if len(word) > 1]
    return tokens

TweetsJoined = ' '.join(train_x) 
TweetsJoined = clean_doc(TweetsJoined)
TweetsJoined = ' '.join(TweetsJoined)
text_file = open(r'C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\Data\\AirlinesTest.txt', 'w')
text_file.write(TweetsJoined)
text_file.close()

In [ ]:
# load the document
filename = 'C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\Data\\AirlinesTest.txt' # Must convert all airline tweets into single text file
text = load_doc(filename)

In [ ]:
# load doc and add to vocab # use this function on txt file of ALL airline tweets
def add_doc_to_vocab(filename, vocab):
    # load doc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # update counts
    vocab.update(tokens)

In [ ]:
vocab = Counter()
add_doc_to_vocab(filename,vocab)
# print the size of the vocab
print(len(vocab))
# print the top words in the vocab
print(vocab.most_common(50))
min_occurane = 1
tokens = [k for k,c in vocab.items() if c >= min_occurane]
#print(len(tokens))

In [ ]:
def save_list(lines, filename):
    # convert lines to a single blob of text
    data = '\n'.join(lines)
    # open file
    file = open(filename, 'w')
    # write text
    file.write(data)
    # close file
    file.close()

# save tokens to a vocabulary file
save_list(tokens, 'C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\Data\\vocab.txt')

In [ ]:
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# load the vocabulary
vocab_filename = 'C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\Data\\vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

In [ ]:
################# Training model ##################

In [ ]:
def clean_doc2(doc, vocab):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    #table = str.maketrans('', '', punctuation)
    #tokens = [w.translate(table) for w in tokens]
    # filter out tokens not in vocab
    tokens = [w for w in tokens if w in vocab]
    tokens = ' '.join(tokens)
    return tokens
CleanedTweets = []
for tweet in train_x:
    Cleaned = clean_doc2(tweet,vocab)
    CleanedTweets.append(Cleaned)
train_docs = CleanedTweets

In [ ]:
from keras.preprocessing.text import Tokenizer
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(train_docs)
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(train_docs)

In [ ]:
from keras_preprocessing.sequence import pad_sequences
# pad sequences
max_length2 = max([len(s.split()) for s in ShuffledTweets])
Xtrain = pad_sequences(encoded_docs, maxlen=max_length2, padding='post')

In [ ]:
train_Y = []
for value in train_y:
    if value == "positive":
        train_Y.append(value.replace("positive","1"))
        
    else:
        train_Y.append(value.replace("negative","0"))
test_Y = []
for value in test_y:
    if value == "positive":
        test_Y.append(value.replace("positive","1"))
        
    else:
        test_Y.append(value.replace("negative","0"))

In [ ]:
test_docs = test_x
# sequence encode
encoded_docs = tokenizer.texts_to_sequences(test_docs)
# pad sequences
Xtest = pad_sequences(encoded_docs, maxlen=max_length2, padding='post')
ytrain = []
for value in train_Y:    
    ytrain.append(int(value))
ytrain = np.array(ytrain)

ytest = []
for value in test_Y:    
    ytest.append(int(value))
ytest = np.array(ytest)

In [ ]:
# define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length2))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

In [ ]:
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(Xtrain, ytrain, epochs=10, verbose=2)

In [ ]:
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

In [ ]:
def remove_non_ascii(string):
    return string.encode('ascii', errors='ignore').decode()
# Returns list of predictions (0 = negative, 1 = positive)
def predict_sentiment(tweets, vocab, tokenizer, model):
    # clean
    cleanedTweets = []
    for tweet in tweets:
        tokens = clean_doc(tweet)
        tokens = [w for w in tokens if w in vocab]
        line = ' '.join(tokens)
        line = remove_non_ascii(line)
        cleanedTweets.append(line)
    
    # convert to line
    
    # encode
    #line = tokenizer.fit_on_texts(line)
    encoded_doc = tokenizer.texts_to_sequences(cleanedTweets)
    # padding
    paddedData = pad_sequences(encoded_doc, maxlen=max_length2, padding='post')
    # predict
    pred = model.predict(paddedData, verbose=0)
    y = pred.tolist()
    predictions = []
    for value in y:
        predictions.append(round(value[0]))
    
    return predictions



In [ ]:
################################################################################################

In [ ]:
# Importing data
data1 = pd.read_csv("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\Data\\hashtag_donaldtrump.csv")
data2 = pd.read_csv("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\Data\\hashtag_joebiden.csv")

data = data1.append(data2, ignore_index=True)
# Removing irrelevant columns
data.drop(data.columns[[1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]], axis=1, inplace=True)

#converting all tweets to strings
data['tweet']=data['tweet'].apply(str)

In [ ]:
def preprocess(text):
    preprocessed_text = []
    for t in text.split():
        if len(t) > 1:
            t = '' if t[0] == '@' and t.count('@') == 1 else t
            t = '' if t.startswith('http') else t
            #t = '' if t.startswith('#') else t
        preprocessed_text.append(t)
    return ' '.join(preprocessed_text)
Cleaned_Tweet =[]
for tweet in data["tweet"]:
    Cleaned_Tweet.append(preprocess(tweet))
data["Cleaned_Tweet"] = Cleaned_Tweet




In [ ]:
dates = ["15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","01","02","03","04","05","06","07","08"]

In [ ]:
#Creating new column with hashtags contained in tweet
data['hashtag'] = data['tweet'].apply(lambda x: re.findall(r"#(\w+)", x))
# creating list of all hashtags used
hashtags = []
for i in data['hashtag']:
    hashtags.append(i)

flat_list = [item for sublist in hashtags for item in sublist]  

In [ ]:
def predict(topic, topicdf):
    # Creating dataframe containing Trump related tweets and their creation date        
    topic = []
    topic = pd.DataFrame(topic)
    topic["created"] = created
    topic["tweet"] = tweet

    # Creating new dataframe for Sentiment analysis results sorted by date
    topicdf = []
    topicdf = pd.DataFrame(topicdf)
    for date in dates:
        tweets = []
        for i in range(len(topic["tweet"])):
            if topic["created"][i].startswith(date):
                tweets.append(topic["tweet"][i])
        #Scaling down tweets to only analyse ~10,000
        random.shuffle(tweets)
        tweets = tweets[:1000]
        startTime = time.time()
        predictions = predict_sentiment(tweets,vocab,tokenizer,model) # creating predictions
        executionTime = (time.time() - startTime)
        print('Execution time in seconds: ' + str(executionTime))
        Neg = predictions.count(0)
        Pos = predictions.count(1)
        values = [Neg,Pos]
        topicdf[date] = values
        print(date)
    return topicdf


In [ ]:
# Aggregating tweets based on hashtag and performing sentiment analysis
# Analysis for Trump related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "Trump":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "trump":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "DonaldTrump":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "donaldtrump":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
Trumptotaltweets = predict("Trump","Trumptotaltweets")
# Analysis for Biden related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "Biden":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "biden":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "JoeBiden":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "joebiden":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
Bidentotaltweets = predict("Biden","Bidentotaltweets")
# Analysis for Elections2020 related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "Elections2020":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "USAElections2020":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "USElections2020":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
Elections2020totaltweets = predict("Elections2020","Elections2020totaltweets")            
# Analysis for KamalaHarris related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "KamalaHarris":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "Harris":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "harris":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
KamalaHarristotaltweets = predict("KamalaHarris","KamalaHarristotaltweets")            
# Analysis for MAGA related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "MAGA":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "MAGA2020":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "maga":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
MAGAtotaltweets = predict("MAGA","MAGAtotaltweets")            
# Analysis for COVID19  related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "COVID19":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "covid19":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "coronavirus":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "Coronavirus":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "COVID":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i]) 
COVIDtotaltweets = predict("COVID","COVIDtotaltweets")
# Analysis for TrumpMeltdown  related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "TrumpMeltdown":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "trumpmeltdown":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])         
TrumpMeltdowntotaltweets = predict("TrumpMeltdown","TrumpMeltdowntotaltweets")
# Analysis for Democrats  related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "Democrats":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "democrats":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])                    
Democratstotaltweets = predict("Democrats","Democratstotaltweets")
# Analysis for GOP  related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "GOP":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "GOP2020":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])         
        elif hashtag == "gop":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])         
GOPtotaltweets = predict("GOP","GOPtotaltweets")
# Analysis for Pennsylvania related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "Pennsylvania":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "pennsylvania":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
Pennsylvaniatotaltweets = predict("Pennsylvania","Pennsylvaniatotaltweets")
# Analysis for Obama  related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "Obama":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "obama":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])            
Obamatotaltweets = predict("Obama","Obamatotaltweets")
# Analysis for Michigan related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "Michigan":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "michigan":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i]) 
Michigantotaltweets = predict("Michigan","Michigantotaltweets")            
# Analysis for VoteHimOut related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "VoteHimOut":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "votehimout":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
VoteHimOuttotaltweets = predict("VoteHimOut","VoteHimOuttotaltweets")
# Analysis for CNN related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "CNN":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "cnn":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
CNNtotaltweets = predict("CNN","CNNtotaltweets")            
# Analysis for Rebulicans related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "Republicans":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "republicans":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])            
Rebulicanstotaltweets = predict("Rebulicans","Rebulicanstotaltweets")            
# Analysis for Florida related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "Florida":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "florida":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
Floridatotaltweets = predict("Florida","Floridatotaltweets")
# Analysis for Georgia related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "Georgia":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "georgia":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
Georgiatotaltweets = predict("Georgia","Georgiatotaltweets")
# Analysis for China related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "China":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "china":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
Chinatotaltweets = predict("China","Chinatotaltweets")
# Analysis for FoxNews related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "FoxNews":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "foxnews":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
FoxNewstotaltweets = predict("FoxNews","FoxNewstotaltweets")
# Analysis for Pence related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "Pence":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "pence":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
Pencetotaltweets = predict("Pence","Pencetotaltweets")
# Analysis for TrumpIsLosing related tweets
created = []
tweet = []
for i in range(len(data["tweet"])):
    for hashtag in data["hashtag"][i]:
        if hashtag == "TrumpIsLosing":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
        elif hashtag == "trumpislosing":
            created.append(data["created_at"][i])
            tweet.append(data["Cleaned_Tweet"][i])
TrumpIsLosingtotaltweets = predict("TrumpIsLosing","TrumpIsLosingtotaltweets")



In [ ]:
Topics = [TrumptotaltweetsCNN,BidentotaltweetsCNN,Elections2020totaltweetsCNN,KamalaHarristotaltweetsCNN,MAGAtotaltweetsCNN,
          COVIDtotaltweetsCNN,TrumpMeltdowntotaltweetsCNN,DemocratstotaltweetsCNN,GOPtotaltweetsCNN,PennsylvaniatotaltweetsCNN,
          ObamatotaltweetsCNN,MichigantotaltweetsCNN,VoteHimOuttotaltweetsCNN,CNNtotaltweetsCNN,RepublicanstotaltweetsCNN,
          FloridatotaltweetsCNN,GeorgiatotaltweetsCNN,ChinatotaltweetsCNN,FoxNewstotaltweetsCNN,PencetotaltweetsCNN,TrumpIsLosingtotaltweetsCNN]
#Calculating sentiment score for each topic sorted by date
dates = ["15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","01","02","03","04","05","06","07","08"]
#Calculating sentiment score for each topic sorted by date
FinalScores =[]
for topic in Topics:    
    Scores = []
    for date in dates:
        SentiScore = topic[date][1]-topic[date][0]
        SentiScore = SentiScore/(topic[date][1]+topic[date][0]) 
        Scores.append(SentiScore)
    FinalScores.append(Scores)

In [ ]:
#saving data
TrumptotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\Trump_CNN.pkl")
BidentotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\Biden_CNN.pkl")
Elections2020totaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\Elections2020_CNN.pkl")
KamalaHarristotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\KamalaHarris_CNN.pkl")
MAGAtotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\MAGA2020_CNN.pkl")
COVIDtotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\COVID_CNN.pkl")
TrumpMeltdowntotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\TrumpMeltdown_CNN.pkl")
DemocratstotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\Democrats_CNN.pkl")
GOPtotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\GOP_CNN.pkl")
PennsylvaniatotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\Pennsylvania_CNN.pkl")
ObamatotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\Obama_CNN.pkl")
MichigantotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\Michigan_CNN.pkl")
VoteHimOuttotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\VoteHimOut_CNN.pkl")
CNNtotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\CNN_CNN.pkl")
RepublicanstotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\Republicans_CNN.pkl")
FloridatotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\Florida_CNN.pkl")
GeorgiatotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\Georgia_CNN.pkl")
ChinatotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\China_CNN.pkl")
FoxNewstotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\FoxNews_CNN.pkl")
PencetotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\Pence_CNN.pkl")
TrumpIsLosingtotaltweetsCNN.to_pickle("C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\SentiData\\TrumpIsLosing_CNN.pkl")

In [ ]:
#Aggregating sentiment data
Topicnames = ["Trump","Biden","Elections2020","KamalaHarris","MAGA",
          "COVID","TrumpMeltdown","Democrats","GOP","Pennsylvania",
          "Obama","Michigan","VoteHimOut","CNN","Republicans",
          "Florida","Georgia","China","FoxNews","Pence","TrumpIsLosing"]
SentimentData = pd.DataFrame(FinalScores[0])
for i in range(0,21):
    SentimentData[i] = FinalScores[i]
index = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
for i in range(0,21):
    int2str = str(index[i])
    SentimentData.rename({index[i]: Topicnames[i]}, axis=1, inplace=True)
SentimentData.rename({21: "TrumpIsLosing"}, axis=1, inplace=True)

In [ ]:
#Saving to csv file for use in R
SentimentData.to_csv('C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\Data\\SentimentDataCNN.csv', index=False)

In [ ]:
SentimentData

In [ ]:
### Performing TweetEval

In [ ]:
# Loading testing data + labels
with open('C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\Data\\TweetEvalTest.txt', encoding="utf8") as f:
    TestData = f.readlines()
with open('C:\\Users\\Jack\\Desktop\\Uni Work\\PROJECT\\Data\\TweetEvalTestLabels.txt', encoding="utf8") as f:
    TestLabels = f.readlines()
TestData_ = []
TestLabels_ = []
for text in TestData:
    TestData_.append(text.replace("\n",""))
for text in TestLabels:
    text.replace("\n","")
    TestLabels_.append(int(text))
TweetEvalTest = []
TweetEvalTest = pd.DataFrame(TweetEvalTest)      
TweetEvalTest["text"] = TestData_
TweetEvalTest["labels"] = TestLabels_
TweetEvalTest = TweetEvalTest[TweetEvalTest.labels != 1]
Labels = []
for value in TweetEvalTest["labels"]:
    if value == 2:
        Labels.append(value-1)
        
    else:
        Labels.append(value)
cleanedTest = []
for text in TweetEvalTest["text"]:
    text.replace("@user","").replace("http","")
    text = text.translate(str.maketrans('', '', string.punctuation))
    cleanedTest.append(text)
    # remove remaining tokens that are not alphabetic
    #tokens = [word for word in tokens if word.isalpha()]
Labels = np.array(Labels)
encoded_doc = tokenizer.texts_to_sequences(cleanedTest)
paddedData = pad_sequences(encoded_doc, maxlen=max_length2, padding='post')

In [ ]:
TweetEvalTest # 0=neg, 1=neutral, 2=pos

In [ ]:
loss, acc = model.evaluate(paddedData, Labels, verbose=0) # testing accuracy for TweetEval for CNN
print('Test Accuracy: %f' % (acc*100))